# Fase 2 - Análise de Dados de Jogos da Steam

Importando bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Carregando e Limpando os dados

In [4]:
# Carregar os dados
df = pd.read_csv('../data/steam_games.csv')

# Visualizando as primeiras linhas para verificar o formato dos dados
print(df.head())

# Verificando informações gerais sobre o DataFrame
print(df.info())

# Removendo linhas com dados faltantes em colunas essenciais
df = df.dropna(subset=['Name', 'Metacritic score', 'Release date', 'Genres', 'DLC count', 'Positive', 'Negative', 'Screenshots', 'Movies', 'Price', 'Publishers', 'Windows', 'Mac', 'Linux'])

# Corrigir valores incorretos (por exemplo, remover espaços extras ou valores inconsistentes)
# df['Windows'] = df['Windows'].str.strip().map({'True': True, 'False': False})
# df['Mac'] = df['Mac'].str.strip().map({'True': True, 'False': False})
# df['Linux'] = df['Linux'].astype(bool)  # Garantindo que 'Linux' seja booleano

# Convertendo colunas relevantes para os tipos corretos
#df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
#df['Metacritic score'] = pd.to_numeric(df['Metacritic score'], errors='coerce')

# Corrigir valores ausentes ou inválidos nas colunas numéricas
#df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0)  # Preencher valores ausentes com 0
#df['Positive'] = pd.to_numeric(df['Positive'], errors='coerce').fillna(0)  # Preencher valores ausentes com 0

# Remover duplicatas, se houver
df = df.drop_duplicates()

# Visualizar as primeiras linhas após a limpeza
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72934 entries, 0 to 72933
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       72934 non-null  int64  
 1   Name                        72932 non-null  object 
 2   Release date                72934 non-null  object 
 3   Estimated owners            72934 non-null  object 
 4   Peak CCU                    72934 non-null  int64  
 5   Required age                72934 non-null  int64  
 6   Price                       72934 non-null  float64
 7   DLC count                   72934 non-null  int64  
 8   About the game              70402 non-null  object 
 9   Supported languages         72934 non-null  object 
 10  Full audio languages        72934 non-null  object 
 11  Reviews                     9217 non-null   object 
 12  Header image                72934 non-null  object 
 13  Website                     354